## Import the neccessary libraries

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

## Moodle Class
The class handles connection to the database and making retrivals.

In [1]:
# The class handles connection to the database and making retrivals on different tables.
class Moodle():
    #constructor
    def __init__(self):
        self.cnx = None
        
    #connection to postgres database
    def connect_db(self,address,port,username,password, name):
        POSTGRES_ADDRESS = address ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
        POSTGRES_PORT = '5432'
        POSTGRES_USERNAME = username ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
        POSTGRES_PASSWORD = password ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
        POSTGRES_DBNAME = name ## CHANGE THIS TO YOUR DATABASE NAME
        
        # A long string that contains the necessary Postgres login information
        postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))
        # Create the connection
        self.cnx = create_engine(postgres_str)
    
    # accepts name of the table and outputs the number of records in it
    def get_num_records_of_table(self, table):
        return pd.read_sql_query('''SELECT COUNT(*) FROM '''+table+''' ;''', self.cnx)
    
    # returns the number of tables in the database   
    def get_num_tables(self):
        return pd.read_sql_query('''select count(*) from information_schema.tables where table_schema = 'public';''', self.cnx)
    
    # returns the number of quiz submissions made in each hour
    def get_num_quiz_submissions_by_hour(self):
        return pd.read_sql_query('''select count(id), EXTRACT(HOUR FROM to_timestamp(timecreated)) AS HOUR from mdl_logstore_standard_log where action='submitted' AND component='mod_quiz' group by EXTRACT(HOUR FROM to_timestamp(timecreated));''', self.cnx)
    
    # returns the number of activities commited by each user of each activity type
    def get_num_events_by_user(self):
        return pd.read_sql_query('''select userid, sum(case when action = 'viewed' then 1 else 0 end) as viewed,sum(case when action = 'uploaded' then 1 else 0 end) as uploaded ,sum(case when action = 'updated' then 1 else 0 end) as updated,sum(case when action = 'answered' then 1 else 0 end) as answered ,sum(case when action = 'abandoned' then 1 else 0 end) as abandoned, sum(case when action = 'submitted' then 1 else 0 end) as Submitted, sum(case when action = 'searched' then 1 else 0 end) as searched, sum(case when action = 'attempted' then 1 else 0 end) as attempted from mdl_logstore_standard_log group by userid;''', self.cnx)
    
    # return the time difference b/n two consecutive clicks
    def get_time_diff(self):
        return pd.read_sql_query('''select userid, action, timecreated, timecreated - lag(timecreated) over (order by userid, timecreated) as time_diff, to_timestamp(timecreated) from mdl_logstore_standard_log; ''',self.cnx)
    # return the information related to users
    def get_user_info(self):
        return pd.read_sql_query('''SELECT id, Country Code ,gender from mdl_user; ''',self.cnx)
    
    # get overall grade of all users
    def get_overall_grades(self):
        return pd.read_sql_query('''SELECT userid, sum(finalgrade) FROM mdl_grade_grades GROUP BY userid ORDER BY userid;''', self.cnx)
    
    # get number of forum posts made by each individual
    def get_num_forum_posts(self):
        return pd.read_sql_query('''SELECT COUNT(*) FROM mdl_forum_posts ;''', self.cnx)
    

## Instansiate Moodle object

In [2]:
moodle = Moodle()

## Make Connection

In [3]:
POSTGRES_ADDRESS = '127.0.0.1'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres'
POSTGRES_PASSWORD = 'nahom' 
POSTGRES_DBNAME = 'Moodle' 
moodle.connect_db(POSTGRES_ADDRESS, POSTGRES_PORT, POSTGRES_USERNAME, POSTGRES_PASSWORD, POSTGRES_DBNAME )

## Session Class
This class is responsible for assigning sessions to every log activity.

In [4]:
# session class
class Session():
    def __init__(self):
        self.session_num = 0
        self.threshold = 200
        self.prev_user = ''
        self.login_time = 0
        self.last_activity_time = 0
        self.total_session_duration = 0
        self.num_sessions = 0
        
    # assing session_num and user id for the incoming row
    def assign_session(self,row):
        if(row['action']=='loggedin'):
            self.session_num += 1
            
        if(row['userid'] != self.prev_user):
            self.session_num = 1
            self.prev_user = row['userid']
        return 'session' + '-' + str(self.session_num)+ '-' + str(row['userid'])
    
#     def get_session_duration():
#         if(row['userid'] != self.prev_user):
#             self.last_activity_time = row['timecreated']
            
#         else:
#             self.first_activity_time = row['timecreated']
#             self.num_sessions = 0
#             self.total_session_duration = 0
#             self.login_time = 0
#             self.prev_user = row['userid']
#     def assign_session(self,row):
        
#         if(row['time_diff'] < 0):
            
#             self.session_num = 1
#             return 'session' + str(self.session_num) + '-' + str(row['userid'])
#         if(row['time_diff'] > self.threshold):
#             self.session_num += 1 
#             return 'session' + str(self.session_num) + '-' + str(row['userid'])

#         return 'session' + str(self.session_num)+ '-' + str(row['userid'])

### Get the time difference b/n two consecutive activities.
The time difference tells us the duration the user spent on each activity.
Negative values mean that particular session is the first ever for that particular user.

In [5]:
# get time difference for consecutive activities
df = moodle.get_time_diff()
df = df[df.userid>0]
df.head(20)

,userid,action,timecreated,time_diff,to_timestamp
9428,1,loggedin,1548435151,-16165465.0,2019-01-25 16:52:31+00:00
9429,1,viewed,1548435152,1.0,2019-01-25 16:52:32+00:00
9430,1,viewed,1548435171,19.0,2019-01-25 16:52:51+00:00
9431,2,loggedin,1548311148,-124023.0,2019-01-24 06:25:48+00:00
9432,2,viewed,1548311179,31.0,2019-01-24 06:26:19+00:00
9433,2,loggedin,1548320487,9308.0,2019-01-24 09:01:27+00:00
9434,2,viewed,1548320488,1.0,2019-01-24 09:01:28+00:00
9435,2,started,1548320489,1.0,2019-01-24 09:01:29+00:00
9436,2,shown,1548320492,3.0,2019-01-24 09:01:32+00:00
9437,2,ended,1548320497,5.0,2019-01-24 09:01:37+00:00


In [6]:
df

,userid,action,timecreated,time_diff,to_timestamp
9428,1,loggedin,1548435151,-16165465.0,2019-01-25 16:52:31+00:00
9429,1,viewed,1548435152,1.0,2019-01-25 16:52:32+00:00
9430,1,viewed,1548435171,19.0,2019-01-25 16:52:51+00:00
9431,2,loggedin,1548311148,-124023.0,2019-01-24 06:25:48+00:00
9432,2,viewed,1548311179,31.0,2019-01-24 06:26:19+00:00
...,...,...,...,...,...
417549,1052,viewed,1563348369,130064.0,2019-07-17 07:26:09+00:00
417550,1052,viewed,1563403999,55630.0,2019-07-17 22:53:19+00:00
417551,1052,viewed,1563783966,379967.0,2019-07-22 08:26:06+00:00
417552,1052,viewed,1563801611,17645.0,2019-07-22 13:20:11+00:00


## Assign Sessions
Apply the assign row function to each row.

In [7]:
# Apply the assign row method of the session class to each row.
session = Session()
df['session'] = df.apply (lambda row: session.assign_session(row), axis=1) # apply awareness to each row

In [8]:
df = df.set_index('userid')

## Engagment Counts
The following cells measure the engagemnt of users based on activity, loggins and dedication time.

### Activity Count

In [9]:
# get the number of activities made by each user
activity_count = activity_count = df.groupby('userid').count()

### Login Count

In [10]:
# get the number of logins made by each user
login_count = df[df.action=='loggedin']

In [11]:
login_count = login_count.groupby('userid').count()
login_count

,action,timecreated,time_diff,to_timestamp,session
userid,,,,,
1,1,1,1,1,1
2,169,169,169,169,169
3,107,107,107,107,107
5,54,54,54,54,54
7,3,3,3,3,3
...,...,...,...,...,...
1028,2,2,2,2,2
1029,1,1,1,1,1
1041,2,2,2,2,2


### Dedication Time
Calculate dedication time by subtracting the timestamp of the login activity from the time stamp of the last activity before the next login

In [12]:
session_start = df.groupby('session')['timecreated'].min() # the timestamp of the first activity (login)
session_end = df.groupby('session')['timecreated'].max() # the timestamp of the last activity (activity before next login)

In [13]:
session_start

session
session-1-1       1548435151
session-1-100     1550331716
session-1-1000    1562595508
session-1-1001    1562595508
session-1-1002    1562581960
                     ...    
session-98-369    1563888735
session-99-2      1556588430
session-99-246    1561721603
session-99-3      1562762879
session-99-369    1563929456
Name: timecreated, Length: 5281, dtype: int64

In [14]:
session_end

session
session-1-1       1548435171
session-1-100     1550331716
session-1-1000    1563801729
session-1-1001    1562595508
session-1-1002    1562835979
                     ...    
session-98-369    1563895182
session-99-2      1556611810
session-99-246    1561724909
session-99-3      1562766591
session-99-369    1563930750
Name: timecreated, Length: 5281, dtype: int64

In [15]:
# get duration by subtraction
durations = session_end - session_start

In [16]:
#change the resulting series to dataframe
durations = durations.to_frame()

In [17]:
durations['session'] = durations.index

### Split the session column to get the session num and user id

In [18]:
durations[['ss','session_num','userid']] = durations.session.str.split('-',expand=True,)
druations = durations.drop('ss',axis=1)

### Change fields to integer type

In [19]:
# rename columns
durations = durations.rename({'timecreated':'duration'})

# change values to int type
durations['userid'] = durations['userid'].astype(int)
durations['session_num'] = durations['session_num'].astype(int)
durations = durations.sort_values(['userid','session_num'])

In [20]:
durations = durations.set_index('userid') # set the index to user id

In [21]:
durations = durations.rename(columns= {'timecreated':'duration'}) # rename the timecreated column to duration
durations

,duration,session,ss,session_num
userid,,,,
1,20,session-1-1,session,1
2,31,session-1-2,session,1
2,960,session-2-2,session,2
2,1,session-3-2,session,3
2,4560,session-4-2,session,4
...,...,...,...,...
1048,921210,session-2-1048,session,2
1049,974860,session-1-1049,session,1
1050,1022252,session-1-1050,session,1


### Get the total Dedication time for each User

In [22]:
dedication = durations.groupby('userid')['duration'].sum() # total dediccation time for each user

#### Change the resulting series to dataframe

In [23]:
# change the resulting series to dataframe 
dedication = dedication.to_frame()
dedication = dedication.rename(columns={'duration':'dedication'})


### View Dedication time of each user

In [24]:
dedication

,dedication
userid,
1,20
2,12296013
3,9313769
4,169
5,9886061
...,...
1048,924971
1049,974860
1050,1022252


### View Login time of each user

In [25]:
login_count

,action,timecreated,time_diff,to_timestamp,session
userid,,,,,
1,1,1,1,1,1
2,169,169,169,169,169
3,107,107,107,107,107
5,54,54,54,54,54
7,3,3,3,3,3
...,...,...,...,...,...
1028,2,2,2,2,2
1029,1,1,1,1,1
1041,2,2,2,2,2


### Combining the the 3 engagement metrics for each user to one table 

In [26]:
activity_count = activity_count.rename(columns={'action':'activity_count'})

dedication = dedication.join(activity_count['activity_count'])

In [27]:
login_count = login_count.rename(columns={'action':'login_count'})
dedication = dedication.join(login_count['login_count'])

## Combined table

In [28]:
dedication

,dedication,activity_count,login_count
userid,,,
1,20,3,1.0
2,12296013,45023,169.0
3,9313769,12922,107.0
4,169,3,NaN
5,9886061,1079,54.0
...,...,...,...
1048,924971,30,1.0
1049,974860,9,NaN
1050,1022252,12,NaN


## Group Students as top 1%, 5%, 10% ...

In [29]:
print('############Top 1%')
print(dedication.quantile(0.99))
print('############Top 5%')
print(dedication.quantile(0.95))
print('############Top 10%')
print(dedication.quantile(0.9))
print('############Top 25%')
print(dedication.quantile(0.75))

############Top 1%
dedication        14067862.25
activity_count        5018.45
login_count             68.05
Name: 0.99, dtype: float64
############Top 5%
dedication        13382474.75
activity_count        1859.50
login_count             29.05
Name: 0.95, dtype: float64
############Top 10%
dedication        13133145.5
activity_count         836.5
login_count             18.1
Name: 0.9, dtype: float64
############Top 25%
dedication        11280940.00
activity_count         123.75
login_count              7.00
Name: 0.75, dtype: float64


## PercentileHandler Class
This class is responsible for identifying in which quantile a user belongs to.

In [30]:
class PercentileHandler():
    # constructor
    def __init__(self,dedication):
        self.dedication = dedication
        
        self.top_1 = dedication.quantile(0.99)
        self.top_5 = dedication.quantile(0.95)
        self.top_10 = dedication.quantile(0.90)
        self.top_25 = dedication.quantile(0.75)
    
    # assign a percentile for each incoming row per column
    def assign_percentile(self,row,column):
        
        if(row[column] >=self.top_1[column]):
            return 1
        elif(row[column] >= self.top_5[column] ):
            return 5
        elif(row[column] >= self.top_10[column] ):
            return 10
        elif(row[column] >= self.top_25[column] ):
            return 25
        else:
            return 100
        

## Assing Percentiles
Call the assign_percentile method for each row (user)

In [31]:
#assign percentile for each user (row)
percentile = PercentileHandler(dedication)
dedication['login_percentile'] = dedication.apply (lambda row: percentile.assign_percentile(row,'login_count'), axis=1)
dedication['activity_percentile'] = dedication.apply (lambda row: percentile.assign_percentile(row,'activity_count'), axis=1)
dedication['dedication_percentile'] = dedication.apply (lambda row: percentile.assign_percentile(row,'dedication'), axis=1)

In [32]:
dedication

,dedication,activity_count,login_count,login_percentile,activity_percentile,dedication_percentile
userid,,,,,,
1,20,3,1.0,100,100,100
2,12296013,45023,169.0,1,1,25
3,9313769,12922,107.0,1,1,100
4,169,3,NaN,100,100,100
5,9886061,1079,54.0,5,10,100
...,...,...,...,...,...,...
1048,924971,30,1.0,100,100,100
1049,974860,9,NaN,100,100,100
1050,1022252,12,NaN,100,100,100


## Get User Information
Get all the necessary information attached with a user like total grades, country and gender

In [33]:
user_info = moodle.get_user_info()

In [34]:
total_grades = moodle.get_overall_grades()

In [35]:
dedication = dedication.join(user_info.set_index('id'))

In [36]:
dedication = dedication.join(total_grades.set_index('userid'))

## Change the total dedication of each user from seconds to hours

In [37]:
dedication['dedication'] = dedication['dedication']/3600

## Replace null values with 0

In [38]:
dedication['login_count'] = dedication['login_count'].fillna(0) # replace null login counts by 0

In [39]:
# replace null values of categorical columns by mode
dedication['code'].fillna((dedication['code'].mode()[0]), inplace=True)
dedication['gender'].fillna((dedication['gender'].mode()[0]), inplace=True)
# replace the null values of numerical columns by 0
dedication['sum'].fillna((0))
dedication['login_count'].fillna((0), )

userid
1         1.0
2       169.0
3       107.0
4         0.0
5        54.0
        ...  
1048      1.0
1049      0.0
1050      0.0
1051      0.0
1052      0.0
Name: login_count, Length: 1046, dtype: float64

## Final Table

In [41]:
dedication

,dedication,activity_count,login_count,login_percentile,activity_percentile,dedication_percentile,code,gender,sum
userid,,,,,,,,,
1,0.005556,3,1.0,100,100,100,Germany,Male,NaN
2,3415.559167,45023,169.0,1,1,25,Germany,Male,NaN
3,2587.158056,12922,107.0,1,1,100,Germany,Male,200.00000
4,0.046944,3,0.0,100,100,100,Germany,Male,NaN
5,2746.128056,1079,54.0,5,10,100,Germany,Male,400.66666
...,...,...,...,...,...,...,...,...,...
1048,256.936389,30,1.0,100,100,100,Nigeria,Male,NaN
1049,270.794444,9,0.0,100,100,100,Ethiopia,Female,NaN
1050,283.958889,12,0.0,100,100,100,Malawi,Male,NaN


### Export to PostGres DB

In [40]:
dedication.to_sql('dedication_table', moodle.cnx, if_exists='replace') # export to postgres